## 1. Initialize data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

In [2]:
data = pd.read_csv('spotify_songs.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40560 entries, 0 to 40559
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           40560 non-null  object 
 1   time_signature     40560 non-null  float64
 2   chorus_hit         40560 non-null  float64
 3   sections           40560 non-null  float64
 4   target             40560 non-null  float64
 5   popularity         312 non-null    float64
 6   sm_target          40560 non-null  float64
 7   tiktok             40560 non-null  int64  
 8   spotify            40560 non-null  int64  
 9   track              40560 non-null  object 
 10  artist             40560 non-null  object 
 11  duration_ms        40560 non-null  float64
 12  danceability       40560 non-null  float64
 13  energy             40560 non-null  float64
 14  key                40560 non-null  object 
 15  loudness           40560 non-null  float64
 16  mode               405

In [4]:
data = data.drop(["popularity", "era", "main_parent_genre", "sections", "chorus_hit" ], axis=1)
data = data.drop(['tiktok', "sm_target", 'spotify', "track"], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40560 entries, 0 to 40559
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          40560 non-null  object 
 1   time_signature    40560 non-null  float64
 2   target            40560 non-null  float64
 3   artist            40560 non-null  object 
 4   duration_ms       40560 non-null  float64
 5   danceability      40560 non-null  float64
 6   energy            40560 non-null  float64
 7   key               40560 non-null  object 
 8   loudness          40560 non-null  float64
 9   mode              40560 non-null  object 
 10  speechiness       40560 non-null  float64
 11  acousticness      40560 non-null  float64
 12  instrumentalness  40560 non-null  float64
 13  liveness          40560 non-null  float64
 14  valence           40560 non-null  float64
 15  tempo             40560 non-null  float64
dtypes: float64(12), object(4)
memory usage: 

In [5]:
#data = pd.get_dummies(data)

In [6]:
data.head()

,track_id,time_signature,target,artist,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1dtKN6wwlolkM8XZy2y9C1,3.0,1.0,Garland Green,173533.0,0.417,0.620,D# / Eb,-7.727,major,0.0403,0.490,0.000000,0.0779,0.845,185.655
1,5hjsmSnUefdUqzsDogisiX,4.0,0.0,Serge Gainsbourg,213613.0,0.498,0.505,D# / Eb,-12.475,major,0.0337,0.018,0.107000,0.1760,0.797,101.801
2,6uk8tI6pwxxdVTNlNOJeJh,4.0,0.0,Lord Melody,223960.0,0.657,0.649,F,-13.392,major,0.0380,0.846,0.000004,0.1190,0.908,115.940
3,7aNjMJ05FvUXACPWZ7yJmv,4.0,0.0,Celia Cruz,157907.0,0.590,0.545,G,-12.058,minor,0.1040,0.706,0.024600,0.0610,0.967,105.592
4,1rQ0clvgkzWr001POOPJWx,4.0,0.0,P. Susheela,245600.0,0.515,0.765,B,-3.515,minor,0.1240,0.857,0.000872,0.2130,0.906,114.617


In [7]:
#Normalize
feature_cols=['acousticness', 'danceability', 'duration_ms', 'energy',
              'instrumentalness', 'liveness', 'loudness', 
              'speechiness', 'tempo', 'time_signature', 'valence',]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_df =scaler.fit_transform(data[feature_cols])

print(normalized_df[:2])

[[0.49196787 0.42206478 0.03811378 0.6199046  0.         0.0658215
  0.78355379 0.04197917 0.76900295 0.6        0.84839357]
 [0.01807229 0.50404858 0.04775985 0.50487572 0.107      0.1653144
  0.69396381 0.03510417 0.42167068 0.8        0.8002008 ]]


In [27]:
data

,track_id,time_signature,target,artist,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1dtKN6wwlolkM8XZy2y9C1,3.0,1.0,Garland Green,173533.0,0.417,0.620,D# / Eb,-7.727,major,0.0403,0.4900,0.000000,0.0779,0.8450,185.655
1,5hjsmSnUefdUqzsDogisiX,4.0,0.0,Serge Gainsbourg,213613.0,0.498,0.505,D# / Eb,-12.475,major,0.0337,0.0180,0.107000,0.1760,0.7970,101.801
2,6uk8tI6pwxxdVTNlNOJeJh,4.0,0.0,Lord Melody,223960.0,0.657,0.649,F,-13.392,major,0.0380,0.8460,0.000004,0.1190,0.9080,115.940
3,7aNjMJ05FvUXACPWZ7yJmv,4.0,0.0,Celia Cruz,157907.0,0.590,0.545,G,-12.058,minor,0.1040,0.7060,0.024600,0.0610,0.9670,105.592
4,1rQ0clvgkzWr001POOPJWx,4.0,0.0,P. Susheela,245600.0,0.515,0.765,B,-3.515,minor,0.1240,0.8570,0.000872,0.2130,0.9060,114.617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40555,4t1TljQWJ6ZuoSY67zVvBI,4.0,0.0,Yolta,150857.0,0.172,0.358,A,-14.430,major,0.0342,0.8860,0.966000,0.3140,0.0361,72.272
40556,2MShy1GSSgbmGUxADNIao5,4.0,1.0,Kodak Black,152000.0,0.910,0.366,C# / Db,-9.954,major,0.0941,0.0996,0.000000,0.2610,0.7400,119.985
40557,55qBw1900pZKfXJ6Q9A2Lc,4.0,1.0,Katy Perry,227760.0,0.719,0.804,A# / Bb,-4.581,major,0.0355,0.0132,0.000003,0.1390,0.6050,119.999
40558,4o9npmYHrOF1rUxxTVH8h4,4.0,0.0,Oscar Peterson,213387.0,0.600,0.177,G,-16.070,major,0.0561,0.9890,0.868000,0.1490,0.5600,120.030


## 2. Get the recommended songs

In [9]:
# extract 10 recommended Songs
indices = pd.Series(data.index, index=data['track_id']).drop_duplicates()
# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)
def generate_recommendation(track_id, model_type=cosine):
    """
    Purpose: Function for song recommendations 
    Inputs: track_id and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index = indices[track_id]
    # Get list of songs for given track_id
    score = list(enumerate(model_type[index]))
    # Sort the most similar songs
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    # Select the top-10 recommended songs
    similarity_score = similarity_score[1:11]
    top_songs_index = [i[0] for i in similarity_score]
    # Top 10 recommended songs
    top_songs = data['track_id'].iloc[top_songs_index]
    return top_songs


In [10]:
#extract 3 recommended Songs
def generate_recommendation(track_id, model_type=cosine):
    index = indices[track_id]
    score = list(enumerate(model_type[index]))
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    similarity_score = similarity_score[1:4]  # Modify to select top 3 recommended songs
    top_songs_index = [i[0] for i in similarity_score]
    top_songs = data['track_id'].iloc[top_songs_index]
    return top_songs

In [11]:
#print(indices.keys())

In [12]:
print("Recommended Songs:")
recommended_songs = generate_recommendation('5hjsmSnUefdUqzsDogisiX',cosine).values
print (recommended_songs)

Recommended Songs:
['1A4eGVWeU3TUd1MxNdxU65' '5Abd8qlHxfliMmkggUCiSY'
 '41TqRBgF4Ahyr4vGmZVk8b']


In [13]:

songs = generate_recommendation ("4t1TljQWJ6ZuoSY67zVvBI")
songs

1428     0clxQCtD1kPfKD1I50t0P4
35846    5EdYF3AG4xOETuvBHukWZn
33766    2TevseTZBCDGTeusyhQXau
Name: track_id, dtype: object

# 3. Convert the track_ids to track_names

In [14]:
# the track id could be in df or tt
df = pd.read_csv('Spotify Data/data-clean.csv')
tt =  pd.read_csv('tiktok.csv')

In [15]:
recommended_songs

array(['1A4eGVWeU3TUd1MxNdxU65', '5Abd8qlHxfliMmkggUCiSY',
       '41TqRBgF4Ahyr4vGmZVk8b'], dtype=object)

In [16]:
def find_track_name(dataframe, track_ids):
    track_names = []
    for track_id in track_ids:
        row = dataframe[dataframe['track_id'] == track_id]
        if not row.empty:
            track_names.append(row['track'].values[0])
        else:
            track_names.append(None)
    return track_names

In [17]:
find_track_name(df, recommended_songs)

['You Got Me Hummin', 'What About Love', 'In-A-Gadda-Da-Vida']

In [18]:
find_track_name(tt, recommended_songs)

[None, None, None]

## 4. Final method to run code

In [20]:
#MOST IMPORTANT CODE
def get_similar_songs (track_id):
    lalala = generate_recommendation (track_id)
    finale = find_track_name(df, lalala)
    return finale

tup = get_similar_songs("7aNjMJ05FvUXACPWZ7yJmv")
tup

["You're Moving Out Today", 'Mango Mangüe', 'Soy Tan Feliz']